# Creación contenedor

## Instalación docker en linux

## Instalación docker en Windows

## Descarga imagen inicial 

## Instalación dependencias básicas

### Actualizar la imagen

### Git

### Jupyter notebook

### Mamba

## Instalación de la arquitectura torchmd-net

### Registar ambiente virtual como ipykernel

## Configuración de funcionalidad